Setting groundtruth and queries

In [5]:
import xml.etree.ElementTree as ET
import matplotlib as plt
import pandas as pd
import numpy as np
import trec
import pprint as pp
import pickle

Qrels = "qrels-clinical_trials.txt"

Queries = "topics-2014_2015-summary.topics"


with open(Queries, 'r') as queries_reader:
    txt = queries_reader.read()

root = ET.fromstring(txt)

cases = {}
for query in root.iter('TOP'):
    q_num = query.find('NUM').text
    q_title = query.find('TITLE').text
    cases[q_num] = q_title


eval = trec.TrecEvaluation(cases, Qrels)


pickle.dump(cases, open("cases.bin", "wb"))


Getting Clinical trials info

In [ ]:
import xml.etree.ElementTree as ET
import tarfile

tar = tarfile.open("clinicaltrials.gov-16_dec_2015.tgz", "r:gz")
doc_ids = []
brief_titles = []
detailed_descriptions = []
brief_summaries = []
criterias = []
genders = []
minimum_ages = []
maximum_ages = []

namelist=tar.getnames()

for tarinfo in tar:
    if tarinfo.size > 500:
        txt = tar.extractfile(tarinfo).read().decode("utf-8", "strict")
        #print(txt)
        root = ET.fromstring(txt)

        judged = False
        for doc_id in root.iter('nct_id'):
            if doc_id.text in eval.judged_docs:
                judged = True
                doc_ids.append(doc_id.text.strip())
        if judged is False:
            continue
       
        brief_title = root.find('brief_title').text.strip()
        brief_titles.append(brief_title)
     
        try:
            detailed_description = root.find('detailed_description').text.strip()
            detailed_descriptions.append(detailed_description)
        except:
            detailed_descriptions.append(brief_title)

        try:
            brief_summary = root.find('brief_summary').text.strip()
            brief_summaries.append(brief_summary)
        except:
            brief_summaries.append(brief_title)
        
        try:
            criteria = root.find('criteria').text.strip
            criterias.append(criteria)
        except:
            criterias.append(brief_title)

"""
        for gender in root.iter('gender'):
            #print(gender)
            genders.append(gender.text.strip())
          

        for minimum_age in root.iter('minimum_age'):
            #print(minimum_age)
            minimum_ages.append(minimum_age.text.strip())
        

        for maximum_age in root.iter('maximum_age'):
            #print(maximum_age)
            maximum_ages.append(maximum_age.text.strip())
"""
tar.close()

print("brief title length: ", len(brief_titles))
print("detailed description length: ", len(detailed_descriptions))
print("brief summary length: ", len(brief_summaries))
print("criteria length: ", len(criterias))
print("genders length", len(genders))
print("minimum_age lenght", len(minimum_ages))
print("maximum_age lenght", len(maximum_ages))


# Aqui criamos os docs pickle para cada uma das partes dos documentos
pickle.dump(doc_ids, open("doc_ids.bin", "wb"))
pickle.dump(brief_titles, open("brief_title.bin", "wb"))
pickle.dump(detailed_descriptions, open("detailed_description.bin", "wb"))
pickle.dump(brief_summaries, open("brief_summary.bin", "wb"))
pickle.dump(criterias, open("criteria.bin", "wb"))
pickle.dump(genders, open("gender.bin", "wb"))
pickle.dump(minimum_ages, open("minimum_age.bin", "wb"))
pickle.dump(maximum_ages, open("maximum_age.bin", "wb"))


Defining abstract class RetrievalModel


In [6]:
import abc  # é preciso importar isto quando queremos definir uma classe abstrata


class RetrievalModel:  # vamos criar uma classe abstrata que é o molde para todas as nossas classes, cada uma um modelo
    @abc.abstractmethod  # para sabermos que RetrievalModel é uma classe abstrata e que, portanto, não pode ser instanciada, ie, "concretizada"
    def search(self):  # aqui nomeia-se uma das funcoes desta classe, neste caso, aquela onde vamos por o codigo q ordenava os docs e ainda classificava a performance do nosso modelo (junto para nao termos q mudar tanto o codigo)
        pass  # nao se pode por nada aqui na abstrata, apenas em cada classe "filho" é que se define a função, aqui apenas se nomeia


VSM Unigram


In [7]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer


class VSM(RetrievalModel):  # definimos a classe de um dos modelos e pomos o RetrievalModel para dizer q esta classe é uma subclasse da classe abstrata

    def search(self, caseid, docs):  # aqui definimos a funcao que faz tudo o q o nosso modelo fazia, pus o codigo ca dentro, pus self.doc em vez de docs
        index = TfidfVectorizer(ngram_range=(
            1, 1), analyzer='word', stop_words=None)
        index.fit(docs)
        X = index.transform(docs)
        query = cases[caseid]
        query_tfidf = index.transform([query])
        doc_scores = 1-pairwise_distances(X, query_tfidf, metric='cosine')
        scores = doc_scores.tolist()
        return scores


LMJM Unigram


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import pairwise_distances
from collections import Counter


class LMJM(RetrievalModel):
    def search(self, caseid, docs):
        index = CountVectorizer(ngram_range=(1, 1), analyzer='word')
        X = index.fit(docs)
        corpus_cv = index.transform(docs)
        all_scores = []
        lmbd = 1
        prob_word_docs = corpus_cv/np.sum(corpus_cv, axis=1)  # p(t|md)
        prob_word_corpus = np.sum(corpus_cv, axis=0) / \
            np.sum(corpus_cv)  # p(t|mc)
        log_mixture = np.log(lmbd*prob_word_docs + (1-lmbd)*prob_word_corpus)
        query = cases[caseid]
        query_cv = index.transform([query])
        total = log_mixture*query_cv.T
        return total


Import relevant documents and libraries

In [9]:
import pickle
import trec
import numpy as np

# Aqui abrimos cada documento pickle e damos-lhes os nomes para usar nas funcoes seguintes
ids = pickle.load(open("doc_ids.bin", "rb"))
brief_title = pickle.load(open("brief_title.bin", "rb"))
detailed_description = pickle.load(open("detailed_description.bin", "rb"))
brief_summary = pickle.load(open("brief_summary.bin", "rb"))
criteria = pickle.load(open("criteria.bin", "rb"))
gender = pickle.load(open("gender.bin", "rb"))
minimum_age = pickle.load(open("minimum_age.bin", "rb"))
maximum_age = pickle.load(open("maximum_age.bin", "rb"))
cases = pickle.load(open("cases.bin", "rb"))


Setting models and text fields

In [10]:
#definir modelos e campos
models = [VSM()]
#, LMJM()
fields = [brief_title, detailed_description, brief_summary, criteria]

Separate training and test queries

In [19]:

print("cases length: ", len(cases))
cases_training = []
cases_test = []
i = 0
k = 12
for caseid in cases:
    if i <= 11:
        cases_test.append(caseid)
    else:
        cases_training.append(caseid)
    i += 1

print("training queries length: ", len(cases_training))
print("test queries length: ", len(cases_test))

cases length:  60
training queries length:  48
test queries length:  12


Getting scores for pairs query_doc for each model and text field

In [12]:
# buscar listas
queries_training = []
docs_training = []
VSM_bt_training = []
VSM_dd_training = []
VSM_bs_training = []
VSM_cr_training = []
LMJM_bt_training = []
LMJM_dd_training = []
LMJM_bs_training = []
LMJM_cr_training = []
y_training = []

for caseid in cases_training:
    case_rel = []
    field_ind = 0
    aux = eval.relevance_judgments.loc[eval.relevance_judgments['query_id'] == int(
        caseid)]
    docs = aux['docid'].tolist()
    #print(len(docs))
    relevances = aux['rel'].tolist()
    for rel in relevances:
        if rel == 0:
            y_training.append(rel)
        elif rel == 1 or rel == 2:
            y_training.append(1)
    #print(len(relevances))
    for docid in docs:
        case_rel.append(ids.index(docid))
    #print(len(case_rel))
    for model in models:
        for field in fields:
            scores = model.search(caseid, field)
            for rel in case_rel:
                value = scores[rel] 
                if field_ind == 0:
                    queries_training.append(caseid)
                    docs_training.append(rel)
                    VSM_bt_training.append(value[0])
                elif field_ind == 1:
                    VSM_dd_training.append(value[0])
                elif field_ind == 2:
                    VSM_bs_training.append(value[0])
                elif field_ind == 3:
                    VSM_cr_training.append(value[0])
                elif field_ind == 4:
                    LMJM_bt_training.append(value[0])
                elif field_ind == 5:
                    LMJM_dd_training.append(value[0])
                elif field_ind == 6:
                    LMJM_bs_training.append(value[0])
                elif field_ind == 7:
                    LMJM_cr_training.append(value[0])
            field_ind += 1
print(len(VSM_bt_training))
print(len(VSM_dd_training))
print(len(VSM_bs_training))
print(len(VSM_cr_training))
print(len(y_training))

print("VSM_brief_title_training: ", len(VSM_bt_training))
print("VSM_detailed_description_training: ", len(VSM_dd_training))
print("VSM_brief_summary_training: ", len(VSM_bs_training))
print("VSM_criteria_training: ", len(VSM_cr_training))
"""
print("LMJM_brief_title_training: ", len(LMJM_bt_training))
print("LMJM_detailed_description_training: ", len(LMJM_dd_training))
print("LMJM_brief_summary_training: ", len(LMJM_bs_training))
print("LMJM_criteria_training: ", len(LMJM_cr_training))
"""
print("y_training: ", len(y_training))


2920
2920
2920
2920
2920


Logistic Regression cross validation to find best C

In [ ]:
# Calcular coeficientes da logistic regression

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

candidates = {'query': queries_training,
              'doc': docs_training,
              'VSM_bt': VSM_bt_training,
              'VSM_dd': VSM_dd_training,
              'VSM_bs': VSM_bs_training,
              'VSM_cr': VSM_cr_training,
              'Y': y_training
              }
df = pd.DataFrame(candidates, columns=['query', 'doc',
                                       'VSM_bt', 'VSM_dd', 'VSM_bs', 'VSM_cr', 'Y'])

x = df[['VSM_bt', 'VSM_dd', 'VSM_bs', 'VSM_cr']]
y = df['Y']

c_values = [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2]
c_ap = []
k = 5
kf = KFold(n_splits=k, random_state=None)

for c in c_values:
    acc_score = []
    for train_index, test_index in kf.split(x):
        X_train, X_test = x.iloc[train_index, :], x.iloc[test_index, :]
        y_train, y_test = y[train_index], y[test_index]
        clf = LogisticRegression(
            random_state=0, C=c, class_weight='balanced').fit(X_train, y_train)
        pred_values = clf.predict(X_test)

        acc = accuracy_score(pred_values, y_test)
        acc_score.append(acc)
    avg_acc_score = sum(acc_score)/k
    c_ap.append(avg_acc_score)

plt.plot(c_values, c_ap)

Training Logistic Regression with best C value

In [14]:
clf = LogisticRegression(
    random_state=0, C=0.3, class_weight='balanced').fit(x, y)
coefs = clf.coef_[0]
print('Coefs: {}'.format(coefs))

Coefs: [-0.33623517 -1.60532446 -0.29509851 -0.33623517]


Eval scores for docs that are judged for a given test query:

In [17]:
p10_list=[]
recall_list=[]
ap_list=[]
ndcg5_list=[]
mrr_list=[]

for caseid in cases_test:
    VSM_bt_test = []
    VSM_dd_test = []
    VSM_bs_test = []
    VSM_cr_test = []
    """
    LMJM_bt_test = []
    LMJM_dd_test = []
    LMJM_bs_test = []
    LMJM_cr_test = []
    """
    y_test = []
    case_rel = []
    field_ind = 0
    zs = []
    aux = eval.relevance_judgments.loc[eval.relevance_judgments['query_id'] == int(
        caseid)]
    docs = aux['docid'].tolist()
    relevances = aux['rel'].tolist()
    for rel in relevances:
        if rel == 0:
            y_test.append(rel)
        elif rel == 1 or rel == 2:
            y_test.append(1)
    for docid in docs:
        case_rel.append(ids.index(docid))
    for model in models:
        for field in fields:
            scores = model.search(caseid, field)
            for rel in case_rel:
                value = scores[rel]
                if field_ind == 0:
                    VSM_bt_test.append(value[0])
                elif field_ind == 1:
                    VSM_dd_test.append(value[0])
                elif field_ind == 2:
                    VSM_bs_test.append(value[0])
                elif field_ind == 3:
                    VSM_cr_test.append(value[0])
                """
                elif field_ind == 4:
                    LMJM_bt_test.append(value[0])
                elif field_ind == 5:
                    LMJM_dd_test.append(value[0])
                elif field_ind == 6:
                    LMJM_bs_test.append(value[0])
                elif field_ind == 7:
                    LMJM_cr_test.append(value[0])
                """
            field_ind += 1
    for line in range(0, len(VSM_bt_test)):
        z = coefs[0]*VSM_bt_test[line]+coefs[1]*VSM_dd_test[line] + coefs[2]*VSM_bs_test[line]+coefs[3]*VSM_cr_test[line]
        zs.append(z)

    doc_ids=[]
    for pos in case_rel:
        doc_ids.append(ids[pos])

    cand={'_id': doc_ids, 'score': zs}
    results = pd.DataFrame(cand, columns = ['_id', 'score'])
    results.sort_values(by=['Z'], inplace=True, ascending=True)
    [p10, recall, ap, ndcg5, mrr] = eval.eval(results, caseid)

    p10_list += [p10]
    recall_list += [recall]
    ap_list += [ap]
    ndcg5_list += [ndcg5]
    mrr_list += [mrr]

print("P10", np.mean(p10_list))
print("Recall", np.mean(recall_list))
print("AP", np.mean(ap_list))
print("NDCG5", np.mean(ndcg5_list))
print("MRR", np.mean(mrr_list))

0.275
1.0
0.30758627981857345
0.18429297351316887
0.24984278055123996


Eval scores for all docs given test queries